In [1]:
from scipy import ndimage, signal, spatial
from scipy.ndimage import morphology

import numpy as np
import pandas as pd
import pdal

In [2]:
data_in = "E:/IG/Msimbazi_odm_georeferenced_model_cropped.las"
p = pdal.Reader.las(data_in).pipeline() | pdal.Filter.sample(radius=1.0).pipeline()
n_points = p.execute()
f'Pipeline selected {n_points} points'

'Pipeline selected 532300 points'

In [3]:
cls = p.arrays[0]['Classification']
cls.fill(1)

In [4]:
df3D = pd.DataFrame(p.arrays[0], columns=['X','Y','Z'])

In [5]:
S = 20
k = 0.000
n = 0.1
b = -0.2

In [6]:
density = n_points / (p.arrays[0]['Y'].ptp() * p.arrays[0]['X'].ptp())
hres = 1. / density

In [7]:
xi = np.ogrid[p.arrays[0]['X'].min():p.arrays[0]['X'].max():hres]
yi = np.ogrid[p.arrays[0]['Y'].min():p.arrays[0]['Y'].max():hres]

In [8]:
bins = df3D.groupby([np.digitize(p.arrays[0]['X'], xi), np.digitize(p.arrays[0]['Y'], yi)])

In [9]:
zmins = bins.Z.min()
cz = np.empty((yi.size, xi.size))
cz.fill(np.nan)
for name, val in zmins.iteritems():
    cz[name[1]-1, name[0]-1] = val

In [10]:
def idw(data):
    # Find indices of the ground returns, i.e., anything that is not a nan, and create a KD-tree.
    # We will search this tree when looking for nearest neighbors to perform the interpolation.
    valid = np.argwhere(~np.isnan(data))
    tree = spatial.cKDTree(valid)
    
    # Now find indices of the non-ground returns, as indicated by nan values. We will interpolate
    # at these locations.
    nans = np.argwhere(np.isnan(data))    
    for row in nans:
        d, idx = tree.query(row, k=12)
        d = np.power(d, -2)
        v = data[valid[idx, 0], valid[idx, 1]]
        data[row[0], row[1]] = np.inner(v, d)/np.sum(d)
        
    return data

In [11]:
cz = idw(cz)

In [12]:
struct = ndimage.iterate_structure(ndimage.generate_binary_structure(2, 1), 11).astype(int)
opened = morphology.grey_opening(cz, structure=struct)

In [13]:
struct = ndimage.iterate_structure(ndimage.generate_binary_structure(2, 1), 9).astype(int)
closed = morphology.grey_closing(opened, structure=struct)

In [14]:
lowx, lowy = np.where((closed - cz) >= 1.0)
cz[lowx, lowy] = closed[lowx, lowy]

In [15]:
stdev = 14
G = np.outer(signal.gaussian(113,stdev), signal.gaussian(113,stdev))
low = signal.fftconvolve(np.pad(cz,2*stdev,'edge'), G, mode='same')[2*stdev:-2*stdev,2*stdev:-2*stdev]/1000.

In [16]:
high = cz - low

In [17]:
erosions = []
granulometry = []
erosions.append(morphology.grey_erosion(high, size=3))
for scale in range(1, S):
    erosions.append(morphology.grey_erosion(erosions[scale-1], size=3))
for scale in range(1, S+1):
    granulometry.append(morphology.grey_dilation(erosions[scale-1], size=2*scale+1))

In [18]:
out = []
for i in range(1, len(granulometry)):
    out.append(granulometry[i-1]-granulometry[i])

In [19]:
gprime = np.maximum.reduce(out)
xs, ys = out[0].shape
gstar = np.zeros((xs,ys))
gplus = np.zeros((xs,ys))
for ii in range(0,xs):
    for jj in range(0,ys):
        for kk in range(0,len(out)):
            if out[kk][ii,jj] < gprime[ii,jj]:
                gplus[ii,jj] += out[kk][ii,jj]
            if out[kk][ii,jj] == gprime[ii,jj]:
                gplus[ii,jj] += out[kk][ii,jj]
                gstar[ii,jj] = kk
                break

In [20]:
T = k * gstar + n
Sg = gprime < T

In [21]:
F = cz.copy()
F[np.where(Sg==0)] = np.nan

In [22]:
G = idw(F)

In [23]:
struct = ndimage.iterate_structure(ndimage.generate_binary_structure(2, 1), 1).astype(int)
gradDTM = morphology.grey_dilation(G, structure=struct)

In [24]:
xbins = np.digitize(df3D.X, xi)
ybins = np.digitize(df3D.Y, yi)
nonground = np.where(df3D.Z >= gradDTM[ybins-1, xbins-1]+b)
ground = np.where(df3D.Z < gradDTM[ybins-1, xbins-1]+b)

In [25]:
g = df3D.iloc[ground]
ng = df3D.iloc[nonground]

In [26]:
cls[ground] = 2
len(cls[ground]) #number of ground points

194561

In [27]:
output = p.arrays[0]
output['Classification'] =cls

In [28]:
pipeline = pdal.Filter.range(limits="Classification[2:2]").pipeline(output)
print(pipeline.execute())

194561


In [29]:
pmf_arr = pipeline.arrays[0]
pipeline = pdal.Filter.pmf().pipeline(pmf_arr) | pdal.Filter.range(limits="Classification[2:2]").pipeline()
print(pipeline.execute())

140827


In [31]:
final_out = pipeline.arrays[0]
pipeline = pdal.Writer.las(filename="ground_only.las",).pipeline(final_out)
print(pipeline.execute())

140827
